# OpenCV中的绘图功能
**cv.line()，cv.circle()，cv.rectangle()，cv.ellipse()，cv.putText()**

在上述所有功能中，您将看到一些常见的参数，如下所示：
* img：您要绘制形状的图像
* color：形状的颜色。对于BGR，将其作为元组传递，例如：(255,0,0)对于蓝色。对于灰度，只需传递标量值即可。
* 厚度：线或圆等的粗细。如果对闭合图形（如圆）传递-1 ，它将填充形状。默认厚度= 1
* lineType：线的类型，是否为8连接线，抗锯齿线等。默认情况下，为8连接线。**cv.LINE_AA**给出了抗锯齿的线条，看起来非常适合曲线。

In [13]:
import numpy as np
import cv2 as cv

## 画线
开始，结束坐标

In [14]:
img = np.zeros((512, 512, 3), np.uint8) # 创建黑色图象
cv.line(img, (0,0), (511,511), (255,0,0), 5)
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()

## 画矩形
左上角， 右下角

横轴x 竖轴y

In [15]:
cv.rectangle(img, (384,0), (510,128), (0, 255, 0), 3)
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()

## 画圆
圆心，半径

In [16]:
cv.circle(img, (447,63), 63, (0,0,255), 5)
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()

[![5piBB8.png](https://z3.ax1x.com/2021/10/07/5piBB8.png)](https://imgtu.com/i/5piBB8)

## 画椭圆
* 中心(x,y)，
* (长轴，短轴)
* angle是椭圆沿**顺**时针方向旋转的角度。
* startAngle和endAngle表示从主轴沿顺时针方向测量的椭圆弧的开始和结束。即给出0和360给出完整的椭圆

In [22]:
cv.ellipse(img, (256,256), (100, 50), 0, 0, 180, 255, -1)
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()

[![5pFu5Q.png](https://z3.ax1x.com/2021/10/07/5pFu5Q.png)](https://imgtu.com/i/5pFu5Q)

**注意** 椭圆中的角度参数并不是普通的角度
比如 设置从45°到225°会得到：

[![5pG0KK.png](https://z3.ax1x.com/2021/10/07/5pG0KK.png)](https://imgtu.com/i/5pG0KK)

它度量角度的坐标是根据椭圆的反三角函数（ arc-angles from an ellipse）而不是圆的反三角函数

比如这个问题，画一个椭圆弧，给定椭圆的极值点和所有参数，除了OpenCV的开始和结束角度，我的解决方案是将椭圆转换为多边形，并切割出极值点所在的区域，因此在cv:：ellipse2poly（）之后：
```cpp
void cutOut(const std::vector<cv::Point> & pts,
            std::vector<cv::Point> & poly, 
            const cv::Point2d & ext1, const cv::Point2d & ext2) const
{
    // whole ellipse
    if ( ext1 == cv::Point2d(0.0,0.0)
         && ext2 == cv::Point2d(0.0,0.0) )
    {
        return;
    }

    // get closest points on ellipse to given extremal points
    double min1 = std::numeric_limits<double>::max();
    double min2 = std::numeric_limits<double>::max();
    int min_ind1, min_ind2;
    for( size_t i = 0; i < pts.size(); i++ ) {
        double n1 = cv::norm(cv::Point2d(pts[i].x,pts[i].y) - ext1);
        if ( n1 < min1 ) {
            min_ind1 = i;
            min1 = n1;
        }
        double n2 = cv::norm(cv::Point2d(pts[i].x,pts[i].y) - ext2);
        if ( n2 < min2 ) {
            min_ind2 = i;
            min2 = n2;
        }
    }

    // copy points to poly
    if ( min_ind2 == min_ind1 ) {
        poly = pts;
    }
    if ( min_ind2 < min_ind1 ) {
        poly.insert(poly.begin(), pts.begin()+min_ind1, pts.end());
        poly.insert(poly.end(),pts.begin(), pts.begin()+min_ind2+1);
    } else {
        poly.insert(poly.begin(), pts.begin()+min_ind1, pts.begin()+min_ind2+1);
    }
}
```

## 画多边形
首先需要顶点的坐标。将这些点组成形状为ROWSx(1x2)的数组，其中ROWS是顶点数，并且其类型应为int32 
实际上 如果第三个参数为False，将获得一条连接所有点的折线，而不是闭合形状。 **cv.polylines()**可用于绘制多条线。只需创建要绘制的所有线条的列表，然后将其传递给函数即可。所有线条将单独绘制。与为每条线调用**cv.line**相比，绘制一组线是一种更好，更快的方法。

In [18]:
pts = np.array([[10,5], [20,30], [70,20], [50,10]], np.int32) # 这里是先用arrary建了一个 rows*2的二维矩阵
pts = pts.reshape((-1,1,2))  # 再用reshape成rows*1*2的数组 自动计算第0维长度
cv.polylines(img, [pts], True, (0,255,255))
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()

[![5pu956.png](https://z3.ax1x.com/2021/10/07/5pu956.png)](https://imgtu.com/i/5pu956)

## 添加文本
* 文字数据
* 位置坐标 数据开始的左下角
* 字体类型（检查**cv.putText**文档以获取受支持的字体）
* 字体比例（字体大小）
* 常规的参数，例如颜色，厚度，线条类型等。为了获得更好的外观，建议使用lineType = **cv.LINE_AA**

In [19]:
help(cv.putText)

Help on built-in function putText:

putText(...)
    putText(img, text, org, fontFace, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]]) -> img
    .   @brief Draws a text string.
    .   
    .   The function cv::putText renders the specified text string in the image. Symbols that cannot be rendered
    .   using the specified font are replaced by question marks. See #getTextSize for a text rendering code
    .   example.
    .   
    .   @param img Image.
    .   @param text Text string to be drawn.
    .   @param org Bottom-left corner of the text string in the image.
    .   @param fontFace Font type, see #HersheyFonts.
    .   @param fontScale Font scale factor that is multiplied by the font-specific base size.
    .   @param color Text color.
    .   @param thickness Thickness of the lines used to draw a text.
    .   @param lineType Line type. See #LineTypes
    .   @param bottomLeftOrigin When true, the image data origin is at the bottom-left corner. Otherwise,
   

| Enumerator ||
|----------|------ |
| cv.FONT_HERSHEY_SIMPLEX | normal size sans-serif font |
| cv.FONT_HERSHEY_PLAIN | small size sans-serif font |
| cv.FONT_HERSHEY_DUPLEX | normal size sans-serif font (more complex than FONT_HERSHEY_SIMPLEX) |
| cv.FONT_HERSHEY_COMPLEX | normal size serif font |
| cv.FONT_HERSHEY_TRIPLEX | normal size serif font (more complex than FONT_HERSHEY_COMPLEX) |
| cv.FONT_HERSHEY_COMPLEX_SMALL | smaller version of FONT_HERSHEY_COMPLEX |
| cv.FONT_HERSHEY_SCRIPT_SIMPLEX | hand-writing style font |
| cv.FONT_HERSHEY_SCRIPT_COMPLEX | more complex variant of FONT_HERSHEY_SCRIPT_SIMPLEX |
| cv.FONT_ITALIC | flag for italic font |

In [20]:
font = cv.FONT_HERSHEY_TRIPLEX
cv.putText(img, 'openCV', (10,500), font, 4, (255,255,255), 2, cv.LINE_AA )
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()